In [1]:
import numpy as np
import torch
import pandas as pd

import matplotlib.pyplot as plt

from toy_datasets.rotational_dataset import make_dataset
from fourier_utils import fourier_transform_point_function, inverse_transform, Complex

In [2]:
dataset = make_dataset(30, 120)

In [3]:
dataset

,vertex_id,x,y,theta,h,output_forces,graph_id
0,0,-1.560141,-0.277406,1.922177,2.716790,-10.421956,0
1,1,0.813282,-0.271660,1.634235,2.294716,-4.152044,0
2,2,-0.794831,-1.175477,0.442421,1.722231,-19.360492,0
3,3,-0.034902,0.395842,5.606118,-2.051685,1.761875,0
4,4,1.188325,0.635423,1.256443,-0.869312,7.875110,0
...,...,...,...,...,...,...,...
3595,115,0.546015,-0.925103,4.998615,-1.382494,13.538758,29
3596,116,-0.006958,1.127028,1.457582,0.440862,-19.397192,29
3597,117,0.586489,0.131765,0.042519,0.924657,-1.314786,29
3598,118,-2.253071,-2.846456,5.720240,0.243505,22.871471,29


### Define a pipeline for casting the raw data to a steerable space

In [4]:
fourier_precision = 10

In [7]:
c_outs = []

for row_num, row in enumerate(dataset['theta']):
    c = fourier_transform_point_function(1, row, precision=fourier_precision)
    data = {}
    for i, j in c:
        i = int(i.real)
        data[f'c_{i}'] = j
    c_row = pd.DataFrame(data, index=[row_num])
    c_outs.append(c_row)
coefficients = pd.concat(c_outs)
dataset = dataset.merge(coefficients,
                        left_index=True,
                        right_index=True
                        )

#### Now define a transformer that works on these

In [12]:
class Transformer(torch.nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, y, theta, input_features):
        """

        :param x:
        :return: predicted outputs for every node
        """

        hidden_features = self.hidden_features(input_features)
        attention = self.get_attention_coefficients(x, y, theta)
        outputs = attention @ hidden_features

        return outputs

    def get_attention_coefficients(self, x, y, theta):
        """

        :return: an n*n matrix of attention coefficients alpha_ij = alpha(i, j)
        """

In [13]:
class Base_SE2_Transformer(torch.nn.Module):
    """An abstract class! Need to subclass to produce invariants """
    def __init__(self, attention_hidden_units):
        super().__init__()

        self.attention_calculator = torch.nn

    def get_num_invariant_dimensions(self):
        raise NotImplementedError("Implement on subclass")

    def forward(self, x, ys):
        raise NotImplementedError() # TODO

    def get_positional_invariants(self, coords_1, coords_2):
        raise NotImplementedError("Create a subclass to implement this!")

class Euclidean_SE2_Transformer(Base_SE2_Transformer):

    def get_positional_invariant(self, coords_1, coords_2):
        x1, y1, theta1 = coords_1
        x2, y2, theta2 = coords_2

        x_diff = x1 - x2
        y_diff = y1 - y2
        return np.sqrt(x_diff**2 + y_diff**2)

    def get_num_invariant_dimensions(self):
        return 1

class FullyExpressive_SE2_Transformer(Base_SE2_Transformer):

    def get_positional_invariants(self, coords_1, coords_2):
        x1, y1, theta1 = coords_1
        x2, y2, theta2 = coords_2

        x_diff = x1 - x2
        y_diff = y1 - y2
        angle_diff = theta2 - theta1
        return np.sqrt(x_diff**2 + y_diff**2), angle_diff


